In [1]:
import cv2
import numpy as np
import math
import colorsys

In [2]:
def DelaunayDiagram(imgd, subdiv):
    height, width = imgd.shape[:2]

    # ドロネーの三角形
    triangles = subdiv.getTriangleList()
    pols = triangles.reshape(-1, 3, 2)
    inner_pts = np.empty((0, 3, 2))
    
    for pa in pols:
        # フレーム外の点は除外
        cnt = 0
        for p in pa:
            if p[0] < 0 or width <= p[0] :
                break
            if p[1] < 0 or height <= p[1] :
                break
            cnt += 1
        # フレーム内の点は有効
        if cnt == 3:
            inner_pts = np.append(inner_pts, [pa], axis=0)
    cv2.fillPoly(imgd, inner_pts.astype(int), (0, 255, 0), 1)
    #print(inner_pts.astype)
    

In [3]:
def DelaunayDiagram2(imgd, subdiv,array):
    height, width = imgd.shape[:2]

    # ドロネーの三角形
    triangles = subdiv.getTriangleList()
    pols = triangles.reshape(-1, 3, 2)
    inner_pts = np.empty((0, 3, 2))
    
    for pa in pols:
        # フレーム外の点は除外
        cnt = 0
        for p in pa:
            if p[0] < 0 or width <= p[0] :
                break
            if p[1] < 0 or height <= p[1] :
                break
            cnt += 1
        # フレーム内の点は有効
        if cnt == 3:
            inner_pts = np.append(inner_pts, [pa], axis=0)
    
    color_list = []
    #温度の平均値計算
    #for i in range(0,255):
        #color_list.append((i,255,255))
   
    for i in range(len(inner_pts)):
        ave = (array[(int)(inner_pts[i][0][0])][(int)(inner_pts[i][0][1])] + array[(int)(inner_pts[i][1][0])][(int)(inner_pts[i][1][1])] + array[(int)(inner_pts[i][2][0])][(int)(inner_pts[i][2][1])])/3
        color_list.append((ave,255,255))
    
    print(inner_pts.shape)
    
    for i in range(len(inner_pts)):
        cv2.fillPoly(imgd, [inner_pts[i].astype(int)], color_list[i], 1)
    #温度に合わせて色を変える
    

In [4]:
def getRandom2DPoints(width, height, n):
    np.random.seed(202001)
    margin = 10
    entry_pts = np.random.randint(margin, max(height, width), (n*2, 2))

    pts = []
    for i, ep in enumerate(entry_pts):
        if ep[0] < margin or (width-margin) < ep[0]:
            continue
        if ep[1] < margin or (height-margin) < ep[1]:
            continue

        # サークル色をHSVで計算
        # 白っぽいを避けるためHSVで計算
        h = i * math.pi * 2.0 / n
        rgb = colorsys.hsv_to_rgb(h, 1.0, 1.0)
        r = int(rgb[0] * 255)
        g = int(rgb[1] * 255)
        b = int(rgb[2] * 255)
        pts.append([ep[0],ep[1],r])
    return    pts

In [5]:
def main():
    height = 240
    width = 320
    
    #点の生成
    pts = getRandom2DPoints(width,height,50)
    #白紙を作る
    img = np.full((height, width, 3), 255, np.uint8)
    
    rect = (0, 0, width, height)
    subdiv = cv2.Subdiv2D(rect)
    #print(pts.shape)
    print(subdiv)
    array = np.zeros((320,240))
    
    for p in pts:
        subdiv.insert((int(p[0]), int(p[1])))
        array[p[0]][p[1]] = p[2]
        #print(p[2])
        
    DelaunayDiagram2(img, subdiv,array)
    
    for p in pts:
        color = p[2]
        cv2.circle(img, (p[0],p[1]), 4, color, thickness=-1, lineType=cv2.LINE_AA)
    print(color)
    cv2.imwrite('test.png', img)
    
    


In [6]:
if __name__ == '__main__':
    main()

<Subdiv2D 0x7fada135ad90>
(108, 3, 2)
220


乱数でデータ作ってドロネー図描くところまでできたから、あとは画像とかネットからデータを拾ってきて作れればいい？